# Imports

In [169]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import ast

# Path and Load Data 

In [59]:
data = pd.read_csv('politics_[actual_size=4968]_processed_merge.csv').politics
data = pd.DataFrame(data=data)
data.head() 

,politics
0,"['scott', 'pruitt', ""trump'"", 'former', 'epa',..."
1,"['joint', 'fundrais', 'committe', 'run', 'marj..."
2,"['earliest', 'day', 'black', 'church', 'polit'..."
3,"['earli', 'novemb', 'rep', 'chip', 'roy', 'tex..."
4,"['add', 'west', 'wing', 'playbook', 'daili', '..."


In [60]:
politics_positive_tweets = data[:4968] 
print('politics tweets size :', len(politics_positive_tweets))
politics_negative_tweets = data[4968:] 
print('non-politics tweets size :', len(politics_negative_tweets))

politics tweets size : 4968
non-politics tweets size : 5000


* Train test split: 20% will be in the test set, and 80% in the training set.


In [61]:
# split the data into two pieces, one for training and one for testing (validation set) 
test_pos = politics_positive_tweets[4000:]
train_pos = politics_positive_tweets[:4000]

test_neg = politics_negative_tweets[4000:]
train_neg = politics_negative_tweets[:4000]

In [77]:
train_x = pd.concat([train_pos, train_neg], axis=0) 
test_x = pd.concat([test_pos, test_neg], axis=0)

- Create the numpy array of positive labels and negative labels.

In [78]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [79]:
print('train shaoe:', train_y.shape)
print('test shape:', test_y.shape)

train shaoe: (8000, 1)
test shape: (1968, 1)


# Extract Vocabulary for Politics Dataset 

In [198]:
def build_freqs(train_x, train_y):
    
    yslist = np.squeeze(train_y).tolist()
    
    train_x_arr = [] 
    for index, row in train_x.iterrows(): 
        for a in row.values: 
            train_x_arr.append(ast.literal_eval(a))
    
    freqs = {}
    for y, tweet in zip(yslist, train_x_arr):
        for word in tweet:
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [199]:
freqs = build_freqs(train_x, train_y) 

In [201]:
len(freqs)

13201